In [1]:
#Setup Data
apikey  = 'QHOA07WHc6ZkNS2A9i4bTcTbYuOpEqLI3eHu6iWp02gDOnmwMFpQsSQDO4nytISY'
secret = 'yvHF7zGHB9CVj8cSHQb0j8cZdjhfNSdJ7FcWJuvIfVlNcCh4aT6UneTJRjcokzH9'

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import warnings
from termcolor import colored
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
    
warnings.filterwarnings('ignore')

#Autenticate
client = Client(apikey, secret)
print("Client initialized successfully")

Client initialized successfully


In [2]:
#Chose to load a Historical Data

import pywt
def seconds_per_interval(interval: str) -> int:
    seconds_per_unit = {
        "s": 1,
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60,
    }
    try:
        return int(interval[:-1]) * seconds_per_unit[interval[-1]] * 1000
    except (ValueError, KeyError):
        return None

data_selections = input("Enter the data selection (e.g., BTCUSDT_15m): ")

file_path = fr'C:\Users\DJKlaKunG\Desktop\Personal_Document\GitHub\CNN-Bi-LSTM-Research\training-data\SWT-CNN-Bi-LSTM-Model-2\{data_selections}_data.csv'

# Split the data_selections string into TICKET and interval
TICKET, interval_str = data_selections.rsplit('_', 1)

# Map the interval string to its corresponding KLINE_INTERVAL attribute
interval_map = {
    '1m': Client.KLINE_INTERVAL_1MINUTE,
    '3m': Client.KLINE_INTERVAL_3MINUTE,
    '5m': Client.KLINE_INTERVAL_5MINUTE,
    '15m': Client.KLINE_INTERVAL_15MINUTE,
    '30m': Client.KLINE_INTERVAL_30MINUTE,
    '1h': Client.KLINE_INTERVAL_1HOUR,
    '2h': Client.KLINE_INTERVAL_2HOUR,
    '4h': Client.KLINE_INTERVAL_4HOUR,
    '6h': Client.KLINE_INTERVAL_6HOUR,
    '8h': Client.KLINE_INTERVAL_8HOUR,
    '12h': Client.KLINE_INTERVAL_12HOUR,
    '1d': Client.KLINE_INTERVAL_1DAY,
    '3d': Client.KLINE_INTERVAL_3DAY,
    '1w': Client.KLINE_INTERVAL_1WEEK,
    '1M': Client.KLINE_INTERVAL_1MONTH,
}

KLINE_INTERVAL = interval_map.get(interval_str, None)
if KLINE_INTERVAL is None:
    print(f"Invalid interval: {interval_str}")
else:
    seconds = seconds_per_interval(KLINE_INTERVAL)
    if seconds is None:
        print(f"Cannot convert interval '{KLINE_INTERVAL}' to milliseconds.")

    
if os.path.exists(file_path):
    print("Found Request file")
    print(f"Requesting {TICKET} - {interval_str}")
    hist_df = pd.read_csv(file_path)
    print(f"Requesting {TICKET} - {interval_str} - Complete")
else:
    print(f"{TICKET} - {interval_str} data is not available")
    print(f"Attempting to retrieve {TICKET} - {interval_str}")
    historical = client.get_historical_klines(TICKET, KLINE_INTERVAL, '1 Jan 2019')
    print(f"Requesting Complete")
    print(f"Sorting Data columes...")
    hist_df = pd.DataFrame(historical)
    hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
    
    hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
    hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
    hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)
    hist_df['Gain'] = hist_df['Close'].diff().clip(lower=0)  # Positive differences only
    hist_df['Loss'] = -hist_df['Close'].diff().clip(upper=0)  # Negative differences only, converted to positive values
    hist_df['Close_T+1'] = hist_df['Close'].shift(-1)
    
    # Ensure the dataset length is even
    hist_df = hist_df.iloc[:-1]
    if len(hist_df) % 2 != 0:
        hist_df = hist_df.iloc[:-1]

    # Initialize the MinMaxScaler
    scaler = MinMaxScaler()
    features_to_transform = ['Open', 'High', 'Low', 'Close', 'Volume', 'Gain', 'Loss']
    # Scale the features and store them in new columns
    for feature in features_to_transform:
        hist_df[f'{feature}_scaled'] = scaler.fit_transform(hist_df[[feature]])

    # Fill any NaN values with 0 for the first row
    hist_df.fillna(0, inplace=True)

    features_scaled = ['Open_scaled', 'High_scaled', 'Low_scaled', 'Close_scaled', 'Volume_scaled', 'Gain_scaled', 'Loss_scaled']
    
    # Check if the data size is appropriate for 2 levels of decomposition
    if len(hist_df) < 10:
        print("Data size is too small for 2 levels of SWT decomposition with db2. Increase the dataset size.")
        # Optionally: exit or continue with other operations
    else:
        # Apply SWT with 'db2' wavelet and level=2, then store the results in new columns
        for feature in features_scaled:
            coeffs = pywt.swt(hist_df[feature], wavelet='db2', level=2, start_level=0)
            
            # For simplicity, we'll take the coefficients of both levels and store them
            for level, (cA, cD) in enumerate(coeffs, 1):
                hist_df[f'{feature}_SWT_level{level}_cA'] = cA
                hist_df[f'{feature}_SWT_level{level}_cD'] = cD


#Preprocessing Data
hist_df.to_csv(file_path, index=False)
print("Preprocessing Data - Complete")

BTCUSDT - 30m data is not available
Attempting to retrieve BTCUSDT - 30m
Requesting Complete
Sorting Data columes...
Preprocessing Data - Complete


In [20]:
#Build Model

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Bidirectional, LSTM, Dense, Dropout, BatchNormalization, concatenate

# Define Input for cA coefficients of Close, Open, High, Low
input_cA = Input(shape=(LOOKBACK, 8))

# CNN-Bi-LSTM Architecture for cA coefficients
x1 = Conv1D(32, 3, activation='relu')(input_cA)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)
x1 = Conv1D(64, 3, activation='relu')(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)
x1 = Bidirectional(LSTM(112, return_sequences=True))(x1)
x1 = Dropout(0.2)(x1)
x1 = Bidirectional(LSTM(112, return_sequences=False))(x1)
x1 = Dropout(0.2)(x1)
x1 = Dense(56, activation='relu')(x1)
x1 = Dropout(0.2)(x1)
x1 = Dense(28, activation='relu')(x1)

# Define Input for Volume, Gain, Loss, and cD coefficients
input_others = Input(shape=(LOOKBACK, 20))

# Bi-LSTM Architecture for the other coefficients
x2 = Bidirectional(LSTM(112, return_sequences=True))(input_others)
x2 = Dropout(0.2)(x2)
x2 = Bidirectional(LSTM(112, return_sequences=False))(x2)
x2 = Dropout(0.2)(x2)
x2 = Dense(56, activation='relu')(x2)
x2 = Dropout(0.2)(x2)
x2 = Dense(28, activation='relu')(x2)

# Merge the two models
merged = concatenate([x1, x2])

# Final dense layers after merging
merged = Dense(56, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(28, activation='relu')(merged)

# Output layer
output = Dense(4, activation='linear')(merged)

# Create the model
model = Model(inputs=[input_cA, input_others], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Model summary
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 5, 8)]               0         []                            
                                                                                                  
 conv1d_8 (Conv1D)           (None, 3, 32)                800       ['input_9[0][0]']             
                                                                                                  
 batch_normalization_8 (Bat  (None, 3, 32)                128       ['conv1d_8[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 dropout_36 (Dropout)        (None, 3, 32)                0         ['batch_normalization_8[